In [1]:
from ortools.linear_solver import pywraplp

### 6. Refinery optimisation

#### Entities

- $C = \{1, 2\}$: types of crude oil.
- $N = \{L, M, H\}$: types of naptha.
- $O = \{L, H, C\}$: types of oil.
- $G = \{C, R\}$: types of gasoline.
- $P  = \{lo, pp, rp, jf, fo\}$: types of output product.

#### Parameters

*Profits*

- $PR_p \, \, \forall \, \, p \in P$: profit we'd make from each output product.

*Ditillation*

- $DN_{cn} \, \, \forall \, \, c \in C, n \in N$: fraction of crude oil $c$ that gets converted to naptha $n$ via distillation.
- $DO_{co} \, \, \forall \, \, c \in C, O \in \{L, H\}$: fraction of crude oil $c$ that gets converted to oil $o$ via distillation.
- $DR_{c} \, \, \forall \, \, c \in C$: fraction of crude oil $c$ that gets converted to residuum via distillation.

*Reforming*

- $R_n \, \, \forall \, \, n \in N$: fraction of each naptha that converts to reformed gasoline.
- $R_{lim} = 10000$: total amount of naptha we can reform every day. 

*Cracking*

- $CO_o \, \, \forall \, \, o \in O$: fraction of each oil that gets cracked to cracked oil.
- $CG_o \, \, \forall \, \, g \in G$: fraction of each oil that gets cracked to cracked gasoline.
- $CL$: fraction of residuum that gets converted to lube oil when cracked. 
- $C_{lim} = 8000$: total amount of oil we can crack per day. 

*Blending*

- $OCT_g \, \, \forall \, \, g \in G$: octane number for each gasoline. 
- $OCT_n \, \, \forall \, \, n \in N$: octane number of each naptha.
- $OCT_{lim, p} \, \, \forall \, \, p \in \{pp, rp\}$: minimum octane number for different petrols.


- $PRS_o \, \, \forall \, \, o \in O$: pressure of each oil.
- $PRS_R$: pressure of residuum.
- $PRS_{lim}$: limit on the pressure of final jet fuel product. 


- $F_o \, \, \forall \, \, o \in O$: ratio (compared to residuum) that must be used to create fuel oil. 

*Limits on input/output volumes*

- $X_{lim, c} \, \, \forall \, \, c \in C$: availability of crude oils.
- $X_{lim} = 45000$: total distillation capacity.
- $l_{lo}, u_{lo}$: lower and upper limit on volume of lube oil we must produce per day. 
- $f_{pp} = 0.4$: minimum fraction of petrol produced that must be premium. 
- $A = 0.5$: fraction of residuum that converts to lube oil.


#### Decision variables

- $X_c \, \, \forall \, \, c \in C$: volume of crude oil to distill.
- $VN_n \, \, \forall \, \, n \in N$: total volume of naptha.
- $VO_o \, \, \forall \, \, o \in O$: total volume of oil.
- $VR$: total volume of residuum.
- $VG_g \, \, \forall \, \, g \in G$: total volume of gasoline.
- $VN_{n, R} \, \, \forall \, \, n \in N$: volume of naptha to reform. 
- $VO_{o, C} \, \, \forall \, \, o \in \{L, H\}$: volume of oil to crack to gasoline. 
- $VR_{lo}$: volume of residuum to crack to lube oil. 
- $VN_{n, p} \, \, \forall \, \, n \in N, p \in \{rp, pp\}$: volume of naptha to put into petrol blend.
- $VG_{g, p} \, \, \forall \, \, g \in G, p \in \{rp, pp\}$: volume of gasoline to put into petrol blend.
- $VO_{o, p} \, \, \forall \, \, o \in O, p \in \{jf, fo\}$: volume of oil to blend.
- $VR_{p} \, \, \forall \, \,p \in \{jf, fo\}$: volume of residuum to blend. 
- $Y_p \, \, \forall \, \, p \in P$: volume of output product.

#### Objective function

$\max(\sum_{p \in P} PR_p \cdot Y_p)$ (maximise profit)


#### Constraints

*Input product limits*

- $\sum_{c \in C} X_{c} \leq X_{lim}$
- $X_c \leq X_{lim, c}$

*Distilling*

- $VN_{n} = \sum_{c \in C} X_c \cdot DN_{cn} \, \, \forall \, \, n \in N$
- $VO_{o} = \sum_{c \in C} X_c \cdot DO_{co} \, \, \forall \, \, o \in \{L, H \}$
- $VR = \sum_{c \in C} X_c \cdot DR_c$ 

*Refining*

- $VG_R = \sum_{n \in N} VN_{n, R} \cdot R_n$
- $\sum_{n \in N} VN_{n , R} \leq R_{lim}$

*Cracking*

- $VO_C = \sum_{o \in L, H} VO_{o, C} \cdot CO_o \, \, \forall \, \, o \in O$
- $VG_C = \sum_{o \in L, H} VO_{o, C} \cdot CG_o \, \, \forall \, \, o \in O$
- $\sum_{o \in O} VO_{o, C} \leq C_{lim}$

*Output*

- $Y_{lo} = CL \cdot VC_{R}$
- $Y_{pp} = \sum_{n \in N} VN_{pp} + \sum_{g \in G} VG_{g,pp}$
- $Y_{rp} = \sum_{n \in N} VN_{rp} + \sum_{g \in G} VG_{g,rp}$
- $Y_{jf} = \sum_{o \in O} VO_{jf} + VR_{jf}$
- $Y_{fo} = \sum_{o \in O} VO_{fo} + VR_{fo}$


- $l_{lo} \leq Y_{lo} \leq u_{lo}$ (limit on output of lube oil)
- $Y_{pp} \geq f_{pp} \cdot Y_{rp}$

*Output product quality*

- $OCT_{lim, pp} \cdot Y_{pp} \leq \sum_{n \in N} OCT_n \cdot VN_{n, pp} + \sum_{g \in G} VG_{g, pp} \cdot OCT_g$
- $OCT_{lim, rp} \cdot Y_{rp} \leq \sum_{n \in N} OCT_n \cdot VN_{n, rp} + \sum_{g \in G} VG_{g, rp} \cdot OCT_g$


- $PRS_{lim} \cdot Y_{jf} \geq \sum_{o \in O} PRS_o \cdot VO_{o, jf} + VR_{jf} \cdot PRS_{R}$


- $VO_{o, fo} = F_o \cdot VR_{fo} \, \, \forall \, \, o \in O$ (fuel oil has specific mix)

*Continuity*

- $VN_{n, pp} + VN_{n, rp} + VN_{n, R} \leq VN_n \, \, \forall \, \, n \in N$
- $VO_{o, jf} + VO_{o, fo} + VO_{o, C} \leq VO_o \, \, \forall o \in \{L, H\}$
- $VO_{C,jf} + VO_{C, fo} \leq VO_{C}$
- $VR_{jf} + VR_{fo} + VR_{lo} \leq VR$
- $VG_{g, pp} + VG_{g, rp} \leq VG_g \, \, \forall \, \, g \in G$

In [2]:
## Entities
C = [1, 2]
N = ["L", "M", "H"]
O = ["L", "H", "C"]
G = ["C", "R"]
P = ["lo", "pp", "rp", "jf", "fo"]

## Parameters

# profits
PR_P = {"pp": 700, "rp": 600, "jf": 400, "fo": 350, "lo": 150}

# distillation
DN_CN = {1: {"L": 0.1, "M": 0.2, "H": 0.2},
         2: {"L": 0.15, "M": 0.25, "H": 0.18}}

DO_CO = {1: {"L": 0.12, "H": 0.2},
         2: {"L": 0.08, "H": 0.19}}

DR_C = {1: 0.13, 2: 0.12}

# reforming
R_N = {"L": 0.6, "M": 0.52, "H": 0.45}
R_lim = 10000

# cracking
CO_O = {"L": 0.68, "H": 0.75}
CG_O = {"L": 0.28, "H": 0.2}
CL = 0.5
C_lim = 8000

# blending
OCT_G = {"C": 105, "R": 115}
OCT_N = {"L": 90, "M": 80, "H": 70}
OCT_lim_P = {"rp": 84, "pp": 94}

PRS_O = {"L": 1., "H": 0.6, "C": 1.5}
PRS_R = 0.05
PRS_lim = 1.

F_O = {"L": 10, "H": 3, "C": 4}

# volume limits
X_lim_C = {1: 20000, 2: 30000}
X_lim = 45000
l_lo, u_lo = 500, 1000
f_pp = 0.4

In [3]:
## Setup solver
solver = pywraplp.Solver("Refinery", 
    pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
inf = solver.infinity()

## Decision variables
X_C = {c: solver.NumVar(lb=0, ub=X_lim_C[c], name=f"X_{c}") for c in C}
VN_N = {n: solver.NumVar(lb=0, ub=inf, name=f"VN_{n}") for n in N}
VO_O = {o: solver.NumVar(lb=0, ub=inf, name=f"VO_{o}") for o in O}
VG_G = {g: solver.NumVar(lb=0, ub=inf, name=f"VG_{g}") for g in G}
VR = solver.NumVar(lb=0, ub=inf, name=f"VR")

VN_NR = {n: solver.NumVar(lb=0, ub=inf, name=f"VN_{n}R") for n in N}
VO_OC = {o: solver.NumVar(lb=0, ub=inf, name=f"VO_{o}C") for o in ["L", "H"]}
VO_OP = {o: {p: solver.NumVar(lb=0, ub=inf, name=f"VO_{o}{p}") 
             for p in ["jf", "fo"]} for o in O}
VR_P = {p: solver.NumVar(lb=0, ub=inf, name=f"VR_{p}") for p in ["jf", "fo"]}
VR_lo = solver.NumVar(lb=0, ub=inf, name="VR_lo")
Y_P = {p: solver.NumVar(lb=0, ub=inf, name=f"Y_{p}") for p in ["jf", "fo", "pp", "rp"]}
Y_P["lo"] = solver.NumVar(lb=l_lo, ub=u_lo, name=f"Y_lo")

VN_NP = {n: {p: solver.NumVar(lb=0, ub=inf, name=f"VN_{n}{p}") 
             for p in ["rp", "pp"]} for n in N}
VG_GP = {g: {p: solver.NumVar(lb=0, ub=inf, name=f"VG_{g}{p}") 
             for p in ["rp", "pp"]} for g in G}

## Objective function
solver.Maximize(solver.Sum(Y_P[p] * PR_P[p] for p in P)) # maximise profits

## Constraints

# input limits
solver.Add(solver.Sum(X_C[c] for c in C) <= X_lim)

# distilling
for n in N:
    solver.Add(VN_N[n] == solver.Sum(X_C[c] * DN_CN[c][n] for c in C))
    
for o in ["L", "H"]:
    solver.Add(VO_O[o] == solver.Sum(X_C[c] * DO_CO[c][o] for c in C))

solver.Add(VR == solver.Sum(X_C[c] * DR_C[c] for c in C))

# refining
solver.Add(VG_G["R"] == solver.Sum(VN_NR[n] * R_N[n] for n in N))
solver.Add(solver.Sum(VN_NR[n] for n in N) <= R_lim)

# cracking
solver.Add(VO_O["C"] == solver.Sum(VO_OC[o] * CO_O[o] for o in ["L", "H"]))
solver.Add(VG_G["C"] == solver.Sum(VO_OC[o] * CG_O[o] for o in ["L", "H"]))
solver.Add(solver.Sum(VO_OC[o] for o in ["L", "H"]) <= C_lim)

# output
solver.Add(Y_P["lo"] == CL * VR_lo)
solver.Add(Y_P["pp"] == solver.Sum(VN_NP[n]["pp"] for n in N) + solver.Sum(VG_GP[g]["pp"] for g in G))
solver.Add(Y_P["rp"] == solver.Sum(VN_NP[n]["rp"] for n in N) + solver.Sum(VG_GP[g]["rp"] for g in G))
solver.Add(Y_P["jf"] == solver.Sum(VO_OP[o]["jf"] for o in O) + VR_P["jf"])
solver.Add(Y_P["fo"] == solver.Sum(VO_OP[o]["fo"] for o in O) + VR_P["fo"])

solver.Add(Y_P["pp"] >= f_pp * Y_P["rp"])


# output quality
for p in ["pp", "rp"]:
    solver.Add(OCT_lim_P[p] * Y_P[p] <= (solver.Sum(OCT_N[n] * VN_NP[n][p] for n in N) 
                                         + solver.Sum(OCT_G[g] * VG_GP[g][p] for g in G)))
    
solver.Add(PRS_lim * Y_P['jf'] >= solver.Sum(PRS_O[o] * VO_OP[o]["jf"] for o in O) + VR_P["jf"] * PRS_R)

for o in O:
    solver.Add(VO_OP[o]["fo"] == F_O[o] * VR_P["fo"])

# continuity
for n in N:
    solver.Add(VN_N[n] >= VN_NP[n]["rp"] + VN_NP[n]["pp"] + VN_NR[n])
    
for o in ["L", "H"]:
    solver.Add(VO_O[o] >= VO_OP[o]["jf"] + VO_OP[o]["fo"] + VO_OC[o])
    
solver.Add(VO_O["C"] >= VO_OP["C"]["jf"] + VO_OP["C"]["fo"])
solver.Add(VR >= VR_P["jf"] + VR_P["fo"] + VR_lo)

for g in G:
    solver.Add(VG_G[g] >= VG_GP[g]["pp"] + VG_GP[g]["rp"])

In [4]:
solver.Solve()
value = int(solver.Objective().Value() / 100)
print(f"Solution found. Optimal policy produces £{value} of profit.")

Solution found. Optimal policy produces £211365 of profit.
